A **Convolutional Neural Network (CNN)** is a specialized type of artificial neural network designed for processing structured grid data, such as images. CNNs are particularly effective in tasks like image classification, object detection, and segmentation because they can automatically and adaptively learn spatial hierarchies of features from input data.

### **How CNNs Work**
1. **Input**:
   - The network takes an image (or structured data) as input, typically represented as a 3D tensor with dimensions `[height, width, channels]`.

2. **Feature Extraction**:
   - The convolutional layers apply filters to detect low-level features like edges.
   - Deeper layers combine these features to detect more complex patterns like shapes or objects.

3. **Downsampling**:
   - Pooling layers reduce the spatial size, making computations efficient while retaining key features.

4. **Classification**:
   - Fully connected layers process the extracted features to make predictions about the input (e.g., whether an image contains a dog, cat, etc.).

---

### **Applications of CNNs**
1. **Object Detection**:
   - E.g., identifying and localizing multiple objects within an image (YOLO, SSD).

2. **Natural Language Processing (NLP)**:
   - Applied to extract features from text data, such as sentiment analysis or text classification.

---

### **Example**
Consider the task of recognizing handwritten digits (like in the MNIST dataset):
1. Input: A 28x28 grayscale image of a digit (e.g., "5").
2. CNN:
   - The first convolutional layer might detect edges.
   - The second layer might detect curves and corners.
   - Deeper layers combine these features to detect the shape of "5".
3. Output: A probability distribution indicating the likelihood of each digit (0–9).


In [6]:
from __future__ import print_function
from sklearn import datasets
import matplotlib.pyplot as plt
import math
import numpy as np

# Import helper functions
import nbimporter


In [ ]:

from neural_network import NeuralNetwork
from utils import train_test_split, to_categorical, normalize
from utils import get_random_subsets, shuffle_data, Plot
from utils.data_operation import accuracy_score
from optimizers import StochasticGradientDescent, Adam, RMSprop, Adagrad, Adadelta
from loss_functions import CrossEntropy
from utils.misc import bar_widgets
from layers import Dense, Dropout, Conv2D, Flatten, Activation, MaxPooling2D, AveragePooling2D, ZeroPadding2D, BatchNormalization, RNN

def main():

    #----------
    # Conv Net
    #----------

    optimizer = Adam()

    data = datasets.load_digits()
    X = data.data
    y = data.target

    # Convert to one-hot encoding
    y = to_categorical(y.astype("int"))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, seed=1)

    # Reshape X to (n_samples, channels, height, width)
    X_train = X_train.reshape((-1,1,8,8))
    X_test = X_test.reshape((-1,1,8,8))

    clf = NeuralNetwork(optimizer=optimizer,
                        loss=CrossEntropy,
                        validation_data=(X_test, y_test))

    clf.add(Conv2D(n_filters=16, filter_shape=(3,3), stride=1, input_shape=(1,8,8), padding='same'))
    clf.add(Activation('relu'))
    clf.add(Dropout(0.25))
    clf.add(BatchNormalization())
    clf.add(Conv2D(n_filters=32, filter_shape=(3,3), stride=1, padding='same'))
    clf.add(Activation('relu'))
    clf.add(Dropout(0.25))
    clf.add(BatchNormalization())
    clf.add(Flatten())
    clf.add(Dense(256))
    clf.add(Activation('relu'))
    clf.add(Dropout(0.4))
    clf.add(BatchNormalization())
    clf.add(Dense(10))
    clf.add(Activation('softmax'))

    print ()
    clf.summary(name="ConvNet")

    train_err, val_err = clf.fit(X_train, y_train, n_epochs=50, batch_size=256)

    # Training and validation error plot
    n = len(train_err)
    training, = plt.plot(range(n), train_err, label="Training Error")
    validation, = plt.plot(range(n), val_err, label="Validation Error")
    plt.legend(handles=[training, validation])
    plt.title("Error Plot")
    plt.ylabel('Error')
    plt.xlabel('Iterations')
    plt.show()

    _, accuracy = clf.test_on_batch(X_test, y_test)
    print ("Accuracy:", accuracy)


    y_pred = np.argmax(clf.predict(X_test), axis=1)
    X_test = X_test.reshape(-1, 8*8)
    # Reduce dimension to 2D using PCA and plot the results
    Plot().plot_in_2d(X_test, y_pred, title="Convolutional Neural Network", accuracy=accuracy, legend_labels=range(10))

if __name__ == "__main__":
    main()